In [1]:
import pandas as pd
import altair as alt

In [2]:
wafer_results = pd.read_csv('../wafer_results_24-11-20_02-42.csv')
particle_results = pd.read_csv('../particle_results_24-11-20_02-42.csv')

wafer_results.replace({'Napoly':'SPT', '2c':'Acrylate', '4c':'Epoxy'}, inplace=True)
particle_results.replace({'Napoly':'SPT', '2c':'Acrylate', '4c':'Epoxy'}, inplace=True)

In [3]:
wafer_results.dropna(inplace=True)  # drop any lines were there were no results
wafer_results.drop(wafer_results[wafer_results.treatment == 'HCl'].index, inplace=True)  # drop HCl treatment (this was only used for calibration)

wafer_results['count_ratios'] = (wafer_results.post_count / wafer_results.pre_count - 1) * 100  # Get normalised count ratios for each wafer: counts of particles in pre state divided by counts in post state. Subtract 1 to normalise. Take absolute value to treat particle loss and particle addition as the same. Multiply by 100 to get percent values.

wafer_polymer_groups = wafer_results.groupby('polymer')  # group by polymers
wafer_results_wrangled =pd.DataFrame()  # create empty df to fill in results from loop
for _, group_content in wafer_polymer_groups:  # cycle through polymer groups
    group_content['count_error'] = group_content['count_ratios'] - group_content.loc[group_content.treatment == 'water', 'count_ratios'].iloc[0]  # subtract count ratio of water from count ratios of all other treatments to get the percentage point error (meaning loss OR addition) of particle numbers caused by each treatment 
    group_content.drop(group_content[group_content.treatment == 'water'].index, inplace=True)  # the rows with the water treatments can now be deleted
    # following line is good example how to not get into "set with copy" warning. Also several bool conditions could be used like that: df.loc[(df['A'] == 'blue') & (df['B'] == 'red') & (df['C'] == 'square'),'D'] = 'M5' (found here: https://stackoverflow.com/questions/21263020/pandas-update-value-if-condition-in-3-columns-are-met)
    #group_content.loc[(group_content['count_error'] < 0), 'count_error'] = 0  # set all ratios that were smaller than water ratio (and thus got now negative due to water ratio correction) to 0.
    wafer_results_wrangled = wafer_results_wrangled.append(group_content)  # save results to df


In [6]:
# useful for radio buttons: https://stackoverflow.com/questions/59025953/on-an-altair-plot-can-you-change-the-location-that-a-selection-e-g-dropdown
# interactive heatmap: https://towardsdatascience.com/altair-plot-deconstruction-visualizing-the-correlation-structure-of-weather-data-38fb5668c5b1

alt.Chart(wafer_results_wrangled).mark_rect().encode(
    x='treatment:N',
    y='polymer:N',
    color='count_error:Q'
)

alt.Chart(...)